In [2]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/furyhawk/text_summarization/raw/master'
    os.system(f'wget {GIT_ROOT}/notebooks/setup.py')

%run -i setup.py

You are working on a local system.
Files will be searched relative to "..".


In [3]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# path to import blueprints packages
sys.path.append(BASE_DIR + '/packages')

In [4]:
# adjust matplotlib resolution
matplotlib.rcParams.update({'figure.dpi': 200 })

In [5]:
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import matplotlib.pyplot as plt
import html
import re
import random
import rouge_score
import wikipediaapi

In [6]:
import requests
from bs4 import BeautifulSoup
import os.path
from dateutil import parser
import pandas as pd
import numpy as np

In [6]:
pd.options.display.max_colwidth = 150 ###
file = "un-general-debates-blueprint.csv"
file = f"{BASE_DIR}/data/un-general-debates/un-general-debates.csv.zip" ### real location
df = pd.read_csv(file)
df.sample(2, random_state=53)

,session,year,country,text
3871,42,1987,HND,"﻿On behalf of the Government of Honduras I extend to you, Sir, sincere congratulations on your election as President of the forty-second session o..."
4697,35,1980,THA,﻿My delegation takes pleasure in extending to Mr. Riidiger von Wechmar its sincere congratulations on his unanimous election to the presidency of ...


# Summarizing text using topic representation
## Identifying important words with TF-IDF values


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
nltk.download('punkt')


sample_text = df['text'].iloc[1]
print("\nSample:", sample_text)
sentences = tokenize.sent_tokenize(sample_text)
tfidfVectorizer = TfidfVectorizer()
words_tfidf = tfidfVectorizer.fit_transform(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\furyx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


Sample: ﻿
May I begin by congratulating you. Sir, on your election to the presidency of the General Assembly at its forty-fourth session. Your wide experience in the United Nations makes you especially qualified to guide the Assembly and its deliberations. You have my best wishes for the demanding task that lies ahead of you. May I also express my thanks to the outgoing President, Mr. Dante Caputo, who so ably fulfilled his role on behalf of the Assembly. It gives me particular pleasure to pay tribute to the Secretary-General. We owe a great deal to his authority and leadership.
As we are assembled here for the annual general debate, every Member state needs to reflect on certain fundamental questions. Is the United Nations doing everything possible in order to meet its challenges? Are we making progress? Can we say that the United Nations enters the last decade of this century with determination and justified self-confidence? Is my own country doing everything it can in order to help

In [8]:
# Parameter to specify number of summary sentences required
num_summary_sentence = 3

# Sort the sentences in descending order by the sum of TF-IDF values
sent_sum = words_tfidf.sum(axis=1)
important_sent = np.argsort(sent_sum, axis=0)[::-1]

# Print three most important sentences in the order they appear in the article
for i in range(0, len(sentences)):
    if i in important_sent[:num_summary_sentence]:
        print (sentences[i])

Both industrialized and developing countries are directly affected by the depletion of the ozone layer through increasing emissions of carbon dioxides, which produce the so-called greenhouse effect in the climate of the Earth, and by the increasing presence of substances in the atmosphere that cause acid rain and direct dangers to health.
At the Conference the representatives of the more than 80 States present agreed unanimously to the Helsinki Declaration on the Protection of  the Ozone Layer, which contains an ambitious undertaking to phase out completely the use of ozone-depleting chlorofluorocarbons by the year 2000, or sooner if possible, and to phase out other ozone-depleting substances as soon as possible.
It is still too early to draw conclusions from the whole process of transition to independence, but we believe that the programme has been brought back on track thanks to the persistent efforts of the Secretary-General and the competent performance of the UNTAG personnel.


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize

def tfidf_summary(text, num_summary_sentence):
    summary_sentence = []
    sentences = tokenize.sent_tokenize(text)
    tfidfVectorizer = TfidfVectorizer()
    words_tfidf = tfidfVectorizer.fit_transform(sentences)
    sentence_sum = words_tfidf.sum(axis=1)
    important_sentences = np.argsort(sentence_sum, axis=0)[::-1]
    for i in range(0, len(sentences)):
        if i in important_sentences[:num_summary_sentence]:
            summary_sentence.append(sentences[i])
    return summary_sentence

In [10]:
from preparation import clean

# Parameter to specify number of summary sentences required
num_summary_sentence = 5

where = df['country'].isin(['SGP'])
sgp_df = data=df[where]
sgp_df.head()

sample_text = clean(df.query("year==2014 and country=='SGP'")['text'].values[0])
test_text = tfidf_summary(sample_text, num_summary_sentence)

print("\ntest text: ", test_text)

,session,year,country,text
64,44,1989,SGP,"﻿First of all, let me congratulate you on your election to the presidency of the General Assembly at its forty-fourth session. Your unique and ill..."
160,25,1970,SGP,"Mr. President, instead of offering you my congratulations as preceding speakers have done, I wish to thank you on behalf of my delegation. We tha..."
240,68,2013,SGP,I warmly \ncongratulate Mr. John Ashe on his election as President \nat the sixty-eighth session of the General Assembly and \nwish him success du...
449,40,1985,SGP,Allow me to congratulate the President on his assumption of this high office. There are few men in this hall who have been associated with the Uni...
661,63,2008,SGP,The collapse of the recent \nDoha talks is deeply troubling. The Doha Development \nRound was the first round of talks to be launched after \nthe ...



test text:  ['Security Council resolution 2178 (2014), on foreign terrorist fighters, which Singapore co-sponsored, is an important step to combat global terrorism and will be critical to cutting off financial and material support for ISIS and preventing the movement of foreign terrorist fighters.', 'Regarding the principles for sustainable development, Singapore is participating actively in the discussions for the post-2015 period, focusing on those areas where we can contribute — key principles that were critical to our development path and might be relevant to other developing countries.', 'As noted on page 17 of the report of the High-level Panel of Eminent Persons on the Post-2015 Development Agenda, cities are where the battle for sustainable development will be won or lost.', 'This year, we will observe that Day together with relevant United Nations agencies and civil society organizations by focusing on the serious problems faced by women and girls who lack basic sanitation ac

NameError: name 'article1' is not defined

In [12]:

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lsa import LsaSummarizer

def lsa_summary(text, num_summary_sentence):
    summary_sentence = []
    LANGUAGE = "english"
    stemmer = Stemmer(LANGUAGE)
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summarizer = LsaSummarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, num_summary_sentence):
        summary_sentence.append(str(sentence))
    return summary_sentence

In [13]:
def download_article(url):
    # check if article already there
    filename = url.split("/")[-1] + ".html"
    filename = f"{BASE_DIR}/notebooks/" + filename
    if not os.path.isfile(filename):
        r = requests.get(url)
        with open(filename, "w+") as f:
            f.write(r.text)
    return filename


In [14]:
def parse_article(article_file):
    with open(article_file, "r") as f:
        html = f.read()
    r = {}
    soup = BeautifulSoup(html, 'html.parser')
    r['id'] = soup.select_one("div.StandardArticle_inner-container")['id']
    r['url'] = soup.find("link", {'rel': 'canonical'})['href']
    r['headline'] = soup.h1.text
    r['section'] = soup.select_one("div.ArticleHeader_channel a").text
    
    r['text'] = soup.select_one("div.StandardArticleBody_body").text
    r['authors'] = [a.text 
                    for a in soup.select("div.BylineBar_first-container.ArticleHeader_byline-bar\
                                          div.BylineBar_byline span")]
    r['time'] = soup.find("meta", { 'property': "og:article:published_time"})['content']
    return r

In [15]:
import reprlib
r = reprlib.Repr()
r.maxstring = 800

url1 = "https://www.reuters.com/article/us-qualcomm-m-a-broadcom-5g/what-is-5g-and-who-are-the-major-players-idUSKCN1GR1IN"
article_name1 = download_article(url1)
article1 = parse_article(article_name1)
print ('Article Published on', r.repr(article1['time']))
print (r.repr(article1['text']))

Article Published on '2018-03-15T11:36:28+0000'
'LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G. A 5G sign is seen at the Mobile World Congress in Barcelona, Spain February 28, 2018. REUTERS/Yves HermanBelow are some facts... 4G wireless and looks set to top the list of patent holders heading into the 5G cycle. Huawei, Nokia, Ericsson and others are also vying to amass 5G patents, which has helped spur complex cross-licensing agreements like the deal struck late last year Nokia and Huawei around handsets. Editing by Kim Miyoung in Singapore and Jason Neely in LondonOur Standards:The Thomson Reuters Trust Principles.'


In [16]:
print(article1['text'])

LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G. A 5G sign is seen at the Mobile World Congress in Barcelona, Spain February 28, 2018. REUTERS/Yves HermanBelow are some facts about 5G and major players. WHAT IS 5G? 5G networks, now in the final testing stage, will rely on denser arrays of small antennas and the cloud to offer data speeds up to 50 or 100 times faster than current 4G networks and serve as critical infrastructure for a range of industries. Deals to start building mass-market 5G networks are still largely a year away, but by 2025, 1.2 billion people are set to have access to 5G networks - a third of them in China, according to the global wireless trade group GSMA. Moving to new networks promises to enable new mobile services and even whole new business m

In [17]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

from sumy.summarizers.lsa import LsaSummarizer

LANGUAGE = "english"
stemmer = Stemmer(LANGUAGE)

parser = PlaintextParser.from_string(article1['text'], Tokenizer(LANGUAGE))
summarizer = LsaSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    print (str(sentence))

LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G.
Moving to new networks promises to enable new mobile services and even whole new business models, but could pose challenges for countries and industries unprepared to invest in the transition.
The concern is that a takeover by Singapore-based Broadcom could see the firm cut research and development spending by Qualcomm or hive off strategically important parts of the company to other buyers, including in China, U.S. officials and analysts have said.
Mobile operators have to upgrade their networks with 5G gear made by the likes of Huawei [HWT.UL] and ZTE 0000630.SZ of China, Sweden’s Ericsson (ERICb.ST) and Finland’s Nokia (NOKIA.HE).
Editing by Kim Miyoung in Singapore and Jason Neely in LondonOur Standards:The Thomson

In [18]:

r.maxstring = 800
url2 = "https://www.reuters.com/article/us-usa-economy-watchlist-graphic/predicting-the-next-u-s-recession-idUSKCN1V31JE"
article_name2 = download_article(url2)
article2 = parse_article(article_name2)
print ('Article Published', r.repr(article1['time']))
print (r.repr(article2['text']))


Article Published '2018-03-15T11:36:28+0000'
'NEW YORK A protracted trade war between China and the United States, the world’s largest economies, and a deteriorating global growth outlook has left investors apprehensive about the end to the longest expansion in American history. FILE PHOTO: Ships and shipping containers are pictured at the port of Long Beach in Long Beach, California, U.S., January 30, 2019.   REUTERS/Mike BlakeThe recent ...hton wrote in the June Cass Freight Index report.  12. MISERY INDEX The so-called Misery Index adds together the unemployment rate and the inflation rate. It typically rises during recessions and sometimes prior to downturns. It has slipped lower in 2019 and does not look very miserable.  Reporting by Saqib Iqbal Ahmed; Editing by Chizu NomiyamaOur Standards:The Thomson Reuters Trust Principles.'


In [19]:

summary_sentence = tfidf_summary(sample_text, num_summary_sentence)
for sentence in summary_sentence:
    print (sentence)

Security Council resolution 2178 (2014), on foreign terrorist fighters, which Singapore co-sponsored, is an important step to combat global terrorism and will be critical to cutting off financial and material support for ISIS and preventing the movement of foreign terrorist fighters.
Regarding the principles for sustainable development, Singapore is participating actively in the discussions for the post-2015 period, focusing on those areas where we can contribute — key principles that were critical to our development path and might be relevant to other developing countries.
As noted on page 17 of the report of the High-level Panel of Eminent Persons on the Post-2015 Development Agenda, cities are where the battle for sustainable development will be won or lost.
This year, we will observe that Day together with relevant United Nations agencies and civil society organizations by focusing on the serious problems faced by women and girls who lack basic sanitation access.
Let us all work to

In [20]:
from sumy.summarizers.text_rank import TextRankSummarizer

parser = PlaintextParser.from_string(sample_text, Tokenizer(LANGUAGE))
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    print (str(sentence))

The President’s choice of the theme “Delivering on and implementing a transformative post-2015 development agenda” for this year’s session and his emphasis on a comprehensive, action-oriented post-2015 development agenda are particularly timely.
Regarding the principles for sustainable development, Singapore is participating actively in the discussions for the post-2015 period, focusing on those areas where we can contribute — key principles that were critical to our development path and might be relevant to other developing countries.
As noted on page 17 of the report of the High-level Panel of Eminent Persons on the Post-2015 Development Agenda, cities are where the battle for sustainable development will be won or lost.
The outcome document of the United Nations Conference on Sustainable Development (resolution 66/288, annex) itself recognizes that well-planned cities promote economically, socially and environmentally sustainable societies.
In 1992, we set up the Singapore Cooperati

In [21]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.text_rank import TextRankSummarizer

def textrank_summary(text, num_summary_sentence):
    summary_sentence = []
    LANGUAGE = "english"
    stemmer = Stemmer(LANGUAGE)
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summarizer = TextRankSummarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, num_summary_sentence):
        summary_sentence.append(str(sentence))
    return summary_sentence

In [22]:
parser = PlaintextParser.from_string(article1['text'], Tokenizer(LANGUAGE))
summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, num_summary_sentence):
    print (str(sentence))

LONDON/SAN FRANCISCO (Reuters) - U.S. President Donald Trump has blocked microchip maker Broadcom Ltd’s (AVGO.O) $117 billion takeover of rival Qualcomm (QCOM.O) amid concerns that it would give China the upper hand in the next generation of mobile communications, or 5G.
Acquiring Qualcomm would represent the jewel in the crown of Broadcom’s portfolio of communications chips, which supply wi-fi, power management, video and other features in smartphones alongside Qualcomm’s core baseband chips - radio modems that wirelessly connect phones to networks.
Qualcomm (QCOM.O) is the dominant player in smartphone communications chips, making half of all core baseband radio chips in smartphones.
Most other baseband chips come from Asia: MediaTek (2454.TW) of Taiwan holds about one quarter of the market, while Samsung Electronics (005930.KS) and Huawei [HWT.UL] - two big smartphone makers - develop chips for their own devices.
Slideshow (2 Images)The standards are set by a global body to ensure a

# Measuring the performance of Text Summarization methods

In [23]:
def print_rouge_score(rouge_score):
    for k,v in rouge_score.items():
        print (k, 'Precision:', "{:.2f}".format(v.precision), 'Recall:', "{:.2f}".format(v.recall), 'fmeasure:', "{:.2f}".format(v.fmeasure))

In [24]:
num_summary_sentence = 3 ##
gold_standard = article2['headline']
summary = ""

summary = ''.join(textrank_summary(article2['text'], num_summary_sentence))
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scores = scorer.score(gold_standard, summary)
print_rouge_score(scores)

rouge1 Precision: 0.06 Recall: 0.83 fmeasure: 0.11


In [30]:
# 

type_of_article = 'business'  # entertainment, politices, sport, tech
root_path = f'../data/BBC News Summary'
num_of_article = len(os.listdir(f"{root_path}/News Articles/{type_of_article}"))

print(f'"Reading {type_of_article} articles"')
df = pd.DataFrame(columns=['title','article','summary'])

for i in tqdm(range(num_of_article)):
    with open(f'{root_path}/News Articles/{type_of_article}/{(i+1):03d}.txt', 'r') as f:
        article = f.read() #.splitlines()
    with open(f'{root_path}/Summaries/{type_of_article}/{(i+1):03d}.txt', 'r') as f:
        summary = f.read() #.splitlines()
    # if len(summary) >=2:
    #     print(i)
    df.loc[i] = [article[0],article[2:],summary]

"Readingbusiness articles"


  0%|          | 0/510 [00:00<?, ?it/s]

In [26]:
df.head()

,title,article,summary
0,A,"sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December,...","TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 200..."
1,D,llar gains on Greenspan speech\n\nThe dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said...,The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to st...
2,Y,kos unit buyer faces loan claim\n\nThe owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back ...,Yukos' owner Menatep Group says it will ask Rosneft to repay a loan that Yugansk had secured on its assets.State-owned Rosneft bought the Yugansk ...
3,H,gh fuel prices hit BA's profits\n\nBritish Airways has blamed high fuel prices for a 40% drop in profits.\n\nReporting its results for the three m...,"Rod Eddington, BA's chief executive, said the results were ""respectable"" in a third quarter when fuel costs rose by Â£106m or 47.3%.To help offset..."
4,P,rnod takeover talk lifts Domecq\n\nShares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a take...,"Pernod has reduced the debt it took on to fund the Seagram purchase to just 1.8bn euros, while Allied has improved the performance of its fast-foo..."


In [27]:
def print_article(article):
    for item in article:
        print(item)

sample_text = clean(df['article'][0])
print( sample_text)

sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's ex

In [28]:
sample_summary = clean(df['summary'][0])
print(sample_summary)

TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations.


In [29]:
num_summary_sentence = 6 ##
gold_standard = sample_summary
summary = ""

summary = ''.join(textrank_summary(sample_text, num_summary_sentence))
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scores = scorer.score(gold_standard, summary)
print_rouge_score(scores)
print(summary)

rouge1 Precision: 0.82 Recall: 0.70 fmeasure: 0.75
sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband.Time Warner's fourth quarter profits were slightly better than analysts' expectations.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.


In [7]:
type_of_article = 'business'  # entertainment, politices, sport, tech
root_path = f'../data/BBC News Summary'
num_of_article = len(os.listdir(f"{root_path}/News Articles/{type_of_article}"))

print(f'"Reading {type_of_article} articles"')
df = pd.DataFrame(columns=['title','article','summary'])

for i in tqdm(range(num_of_article)):
    with open(f'{root_path}/News Articles/{type_of_article}/{(i+1):03d}.txt', 'r') as f:
        article = f.read() #.splitlines()
    with open(f'{root_path}/Summaries/{type_of_article}/{(i+1):03d}.txt', 'r') as f:
        summary = f.read() #.splitlines()
    # if len(summary) >=2:
    #     print(i)
    df.loc[i] = [article[0],article[2:],summary]

"Readingbusiness articles"


100%|██████████| 510/510 [00:22<00:00, 22.19it/s]


In [10]:
def print_article(article):
    for item in article:
        print(item)

sample_text = df['article'][0]
print( sample_text)


 sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL'

In [12]:

#loading BART (bert summarizer)

from transformers import pipeline
# Open and read the article
to_tokenize = sample_text

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")
summarized = summarizer(to_tokenize, min_length=75, max_length=300)

# Print summarized text
print(summarized)



Downloading: 100%|██████████| 1.80k/1.80k [00:00<00:00, 601kB/s]
Downloading: 100%|██████████| 1.22G/1.22G [00:25<00:00, 48.7MB/s]
Downloading: 100%|██████████| 899k/899k [00:01<00:00, 716kB/s]
Downloading: 100%|██████████| 456k/456k [00:01<00:00, 363kB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 13.0kB/s]


[{'summary_text': ' Profits buoyed by one-off gains offset a profit dip at Warner Bros, and less users for AOL . For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from 2003 . But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman .'}]


In [14]:


from bert_score import score
# bert score with full text
P, R, F1 = score([summarized[0]['summary_text']], [sample_text], lang='en', verbose=True)
P,R,F1



Downloading: 100%|██████████| 482/482 [00:00<00:00, 241kB/s]
Downloading: 100%|██████████| 899k/899k [00:01<00:00, 606kB/s]
Downloading: 100%|██████████| 456k/456k [00:01<00:00, 375kB/s]
Downloading: 100%|██████████| 1.36M/1.36M [00:01<00:00, 799kB/s] 
Downloading: 100%|██████████| 1.43G/1.43G [00:28<00:00, 49.8MB/s]
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequence

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:08<00:00,  8.17s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 52.63it/s]

done in 8.20 seconds, 0.12 sentences/sec


(tensor([0.9163]), tensor([0.8220]), tensor([0.8666]))

In [16]:
print("Full text length ",len(sample_text))
print("Full text length ",len(summarized[0]['summary_text']))

Full text length  2558
Full text length  276


In [18]:
print("Full text length ", summarized[0]['summary_text'])


Full text length  {'summary_text': ' Profits buoyed by one-off gains offset a profit dip at Warner Bros, and less users for AOL . For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from 2003 . But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman .'}
